<a href="https://colab.research.google.com/github/SDS-AAU/UNISTRA-DS-2022/blob/master/static/workshops/2021/UNISTRA_w1_job_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install kaggle package
! pip install -q kaggle

In [ ]:
# make folder for api key
! mkdir ~/.kaggle

In [ ]:
# copy key into folder
! cp kaggle.json ~/.kaggle/

In [ ]:
# change access permissions
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# check if worked
! kaggle datasets list

ref                                                              title                                               size  lastUpdated          downloadCount  
---------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
yasserh/wine-quality-dataset                                     Wine Quality Dataset                                21KB  2022-01-15 19:15:11           7916  
mkoklu42/pistachio-dataset                                       Pistachio Dataset                                    2MB  2022-02-11 21:06:50             99  
prasertk/netflix-subscription-price-in-different-countries       Netflix subscription fee in different countries      3KB  2022-01-15 07:06:09           6058  
majyhain/height-of-male-and-female-by-country-2022               Height of Male and Female by Country 2022            4KB  2022-02-02 00:40:19           3273  
ashishjangra27/ted-talks                

In [ ]:
! kaggle datasets download -d kandij/job-recommendation-datasets

 94% 49.0M/52.4M [00:01<00:00, 27.0MB/s]
100% 52.4M/52.4M [00:02<00:00, 27.1MB/s]


In [ ]:
! unzip /content/job-recommendation-datasets.zip

Archive:  /content/job-recommendation-datasets.zip
  inflating: Combined_Jobs_Final.csv  
  inflating: Experience.csv          
  inflating: Job_Views.csv           
  inflating: Positions_Of_Interest.csv  
  inflating: job_data.csv            


In [ ]:
import pandas as pd
import numpy as np
import spacy

#instantiating English module
nlp = spacy.load('en')

In [ ]:
df_jobs = pd.read_csv('/content/Combined_Jobs_Final.csv')
df_views = pd.read_csv('/content/Job_Views.csv')
df_poi = pd.read_csv('/content/Positions_Of_Interest.csv')
df_exp = pd.read_csv('/content/Experience.csv')


In [ ]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84090 entries, 0 to 84089
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Job.ID              84090 non-null  int64  
 1   Provider            84090 non-null  int64  
 2   Status              84090 non-null  object 
 3   Slug                84090 non-null  object 
 4   Title               84090 non-null  object 
 5   Position            84090 non-null  object 
 6   Company             81819 non-null  object 
 7   City                83955 non-null  object 
 8   State.Name          83919 non-null  object 
 9   State.Code          83919 non-null  object 
 10  Address             36 non-null     object 
 11  Latitude            84090 non-null  float64
 12  Longitude           84090 non-null  float64
 13  Industry            267 non-null    object 
 14  Job.Description     84034 non-null  object 
 15  Requirements        0 non-null      float64
 16  Sala

In [ ]:
# concatenate several columns into one text
df_jobs['text'] = df_jobs['Title'].str.cat(df_jobs['Position'].astype(str), sep=' ').str.cat(df_jobs['Company'].astype(str), sep=' ').str.cat(df_jobs['Job.Description'].astype(str), sep=' ')

In [ ]:
# progress bar
import tqdm

In [ ]:
# how long would it take to run it with plain spacy?

%%time
nlp(df_jobs['text'][1])

CPU times: user 72.9 ms, sys: 3.46 ms, total: 76.4 ms
Wall time: 82.9 ms


Kitchen Staff/Chef @ Claude Lane Kitchen Staff/Chef Claude Lane  

New French Brasserie in S.F. Financial District Seeks Chef
We are seeking an energetic, dynamic chef to take charge and grow with our company. Our ideal candidate is a motivated self-starter, has a great work ethic and is ready for the challenge of building their own team with executive support. 

This position is a perfect fit for a talented chef ready to take the next step, someone who can multi task in a high volume kitchen and has exceptional organizational skills. Position requirements are experience with French cuisine, a minimum of 5 years as a sous chef in high volume, full service restaurant. He or she must be a team player, leading by example and working side by side with other members of the team. 

Computer and management skills are a must, as are experience in ordering, inventory and cost control. The right candidate will have previously demonstrated an understanding of leadership and accountability, and an

In [ ]:
# run that for all? That is 0.71h - too long
60*84000/1000/3600

1.4

In [ ]:
# run progress bare and clean up using spacy but without some heavy parts of the pipeline

%%time
clean_text = []


pbar = tqdm.tqdm(total=len(df_jobs['text']),position=0, leave=True)

for text in nlp.pipe(df_jobs['text'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_text.append(txt)

  pbar.update(1)

100%|█████████▉| 84036/84090 [02:38<00:00, 608.50it/s]

CPU times: user 2min 31s, sys: 2.21 s, total: 2min 33s
Wall time: 2min 38s


In [ ]:
df_jobs['clean_text'] = clean_text

In [ ]:
df_jobs['clean_text']

0        [server, tacolicious, server, tacolicious, tac...
1        [kitchen, staff, chef, claude, lane, kitchen, ...
2        [bartender, machka, restaurants, bartender, ma...
3        [server, teriyaki, house, server, teriyaki, ho...
4        [kitchen, staff, chef, rosa, mexicano, sunset,...
                               ...                        
84085    [book, keeper, national, japanese, american, h...
84086    [kitchen, staff, chef, emporio, rulli, kitchen...
84087    [driver, onigilly, driver, onigilly, onigilly,...
84088    [line, cook, machka, restaurants, line, cook, ...
84089    [cashier, kazoo, restaurant, cashier, kazoo, r...
Name: clean_text, Length: 84090, dtype: object

In [ ]:
df_jobs['clean_text'].isnull().sum()

0

In [ ]:
# update gensim
!pip install --upgrade gensim -q

     |████████████████████████████████| 24.1 MB 1.4 MB/s 


In [ ]:
# get tooling for Word2Vec model
from gensim.models import Word2Vec

In [ ]:
# enable logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# train word2vec model
w2v_model = Word2Vec(sentences=df_jobs['clean_text'], vector_size=300, window=5, min_count=2, workers=2, epochs=5)

2022-03-02 09:23:58,319 : INFO : collecting all words and their counts
2022-03-02 09:23:58,332 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-02 09:23:59,112 : INFO : PROGRESS: at sentence #10000, processed 1489501 words, keeping 18094 word types
2022-03-02 09:24:00,211 : INFO : PROGRESS: at sentence #20000, processed 2801776 words, keeping 25505 word types
2022-03-02 09:24:01,793 : INFO : PROGRESS: at sentence #30000, processed 4383902 words, keeping 30826 word types
2022-03-02 09:24:02,893 : INFO : PROGRESS: at sentence #40000, processed 5723039 words, keeping 35286 word types
2022-03-02 09:24:04,111 : INFO : PROGRESS: at sentence #50000, processed 7118565 words, keeping 39058 word types
2022-03-02 09:24:04,879 : INFO : PROGRESS: at sentence #60000, processed 8409519 words, keeping 42530 word types
2022-03-02 09:24:05,753 : INFO : PROGRESS: at sentence #70000, processed 9822031 words, keeping 45756 word types
2022-03-02 09:24:06,588 : INFO : PROGRE

In [ ]:
w2v_model.wv.similar_by_word('sushi')

[('pastry', 0.6883001327514648),
 ('salad', 0.6454644203186035),
 ('chicken', 0.6405736804008484),
 ('fry', 0.6381990909576416),
 ('dessert', 0.6287106871604919),
 ('sous', 0.617621123790741),
 ('bartender', 0.6161249876022339),
 ('soup', 0.6134330034255981),
 ('finisher', 0.6032063364982605),
 ('chefs', 0.6014286875724792)]

In [ ]:
# check out cosine similarity for some word-vectors
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pizza = w2v_model.wv['pizza'].reshape(1,300)
pasta = w2v_model.wv['pasta'].reshape(1,300)
sushi = w2v_model.wv['sushi'].reshape(1,300)
uber = w2v_model.wv['uber'].reshape(1,300)

In [ ]:
cosine_similarity(pizza, uber)

array([[0.13147064]], dtype=float32)

In [ ]:
cosine_similarity(pizza, pasta)

array([[0.3657799]], dtype=float32)

In [ ]:
w2v_model.save('w2v_model')

2022-03-02 10:02:12,720 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'w2v_model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2022-03-02T10:02:12.720010', 'gensim': '4.1.2', 'python': '3.7.12 (default, Jan 15 2022, 18:48:18) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2022-03-02 10:02:12,730 : INFO : not storing attribute cum_table
2022-03-02 10:02:13,076 : INFO : saved w2v_model


In [ ]:
# define function for avg-embeddings 
# (in older versions of gensim it's not key_to_index but vocab)

def get_mean_vector(word2vec_model, words):
    # remove out-of-vocabulary words
    words = [word for word in words if word in w2v_model.wv.key_to_index]
    if len(words) >= 1:
        return np.mean(word2vec_model.wv[words], axis=0)
    else:
        return []

In [ ]:
df_jobs['clean_text'][0]

In [ ]:
get_mean_vector(w2v_model, df_jobs['clean_text'][0])

In [ ]:
# transform all texts into avg-vec-repre
avg_job_vecs = df_jobs['clean_text'].map(lambda t: get_mean_vector(w2v_model, t))

In [ ]:
avg_job_vecs[:10]

0    [-0.38937545, 0.39502305, -0.24858037, -0.5289...
1    [0.008985659, 0.005536845, -0.30841076, -0.731...
2    [0.006986467, -0.0059137023, -0.026989335, -0....
3    [-0.37293038, 0.7175547, -0.30870807, -0.30775...
4    [0.09119703, 0.4008965, 0.34726006, -0.4725212...
5    [0.23849797, 0.06390896, 0.42710426, -0.540001...
6    [-0.1828821, 0.62840605, -0.3854589, -0.557041...
7    [-0.060302176, 0.1282239, -0.047130395, -0.608...
8    [-0.036706682, -0.026592985, 0.4509321, -0.682...
9    [-0.18551093, -0.016812475, -0.20467716, -0.46...
Name: clean_text, dtype: object

In [ ]:
# aggregate vectors from list to matrix
avg_job_vecs = np.vstack(avg_job_vecs)

In [ ]:
avg_job_vecs.shape

(84090, 300)

In [ ]:
# calculate similarity to 1 article
sims = cosine_similarity(avg_job_vecs[0].reshape(1,300), avg_job_vecs)

In [ ]:
sims

array([[1.        , 0.33396965, 0.56535184, ..., 0.47272855, 0.5303309 ,
        0.50748646]], dtype=float32)

In [ ]:
# extract indices
ix = np.flip(np.argsort(sims)).tolist()[0][:10]

In [ ]:
# show results
df_jobs['text'][ix]

0        Server @ Tacolicious Server Tacolicious Tacoli...
84057    Server @ Kabuto Restaurant Server Kabuto Resta...
13448    Server @ BALEENkitchen Server BALEENkitchen  ●...
24471    Server @ Exotic Thai Cafe Server Exotic Thai C...
84081    Server @ Pizza Antica Server Pizza Antica  ● S...
13453    Server @ Gonpachi Server Gonpachi  ● Serve foo...
10783    Server @ La Fontaine Restaurant Server La Font...
84082    Server @ Giardino Server Giardino  ● Serve foo...
84044    Server @ Yuzu Server Yuzu  Yuzu is one of the ...
10778    Server @ Far Niente Ristorante Server Far Nien...
Name: text, dtype: object

In [ ]:
# free text query
query = 'consultant in legal'

query = nlp(query, disable=["tagger", "parser", "ner"])
query = [token.lemma_.lower() for token in query 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

query = get_mean_vector(w2v_model,query)

In [ ]:
sims = cosine_similarity(query.reshape(1,300), avg_job_vecs)
ix = np.flip(np.argsort(sims)).tolist()[0][:10]
df_jobs['text'][ix]

53292    HR Consultant - ACA expert needed! @ OfficeTea...
56218    Leasing Consultant @ Eastwood Management Corpo...
44270    Leasing Consultant Leasing Consultant nan Leas...
37578    Legal Assistant/ Project Assistant @ Carolina ...
76525    Leasing Consultant @ OfficeTeam Leasing Consul...
13691    Administrative Assistant with Leasing Experien...
47978    Paralegal/legal assistant/litigation specialis...
25871    Legal Writer (part-time) @ Randstad Profession...
53752    Senior Consultant   EMCC A2D2 @ GreyStone Staf...
10661    Legal assistant @ Law Offices of Bijan Sebasti...
Name: text, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# function that does absolutely nothing...
# to be able to use TfidfVectorizer on already tokenized text
def dummy_fun(doc):
    return doc

In [ ]:
# we turn of any preprocessing and align vocabulary with the one
# used by our embeddings
# that will allow us to use TFIDF vectors to weight the embeddings

tfidf = TfidfVectorizer(vocabulary=w2v_model.wv.key_to_index.keys(),
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

In [ ]:
# create TFIDF matrix (we could also just use that one for search)
df_jobs_tfidf = tfidf.fit_transform(df_jobs['clean_text'])

In [ ]:
# how many word-vectors do we have?
len(w2v_model.wv.key_to_index)

33071

In [ ]:
# one tfidf vector has also 33071 columns - because we provided a vocab
df_jobs_tfidf[:1,:]

<1x33071 sparse matrix of type '<class 'numpy.float64'>'
	with 27 stored elements in Compressed Sparse Row format>

To get the vectors, we can use the dot-product of the TFIDF vector (or full matrix) with our word embeddings. n-columns (TFIDF) = n-rows (W2V embeddings)

![](https://hadrienj.github.io/assets/images/2.2/dot-product.png)

In [ ]:
# we can use np.dot or since Python 3 the @ for matrix-multiplication

# let's try
df_jobs_tfidf[:1,:] @ w2v_model.wv.vectors

In [ ]:
# for the whole matrix

df_jobs_w2v_tfidf = df_jobs_tfidf @ w2v_model.wv.vectors

In [ ]:
# calculate similarity to 1 article
sims = cosine_similarity(df_jobs_w2v_tfidf[0].reshape(1,300), df_jobs_w2v_tfidf)

# extract indices
ix = np.flip(np.argsort(sims)).tolist()[0][:10]

# show results
df_jobs['text'][ix]

0        Server @ Tacolicious Server Tacolicious Tacoli...
84057    Server @ Kabuto Restaurant Server Kabuto Resta...
13448    Server @ BALEENkitchen Server BALEENkitchen  ●...
84044    Server @ Yuzu Server Yuzu  Yuzu is one of the ...
13453    Server @ Gonpachi Server Gonpachi  ● Serve foo...
54458    Server @ Sakae Sushi Server Sakae Sushi  Locat...
84081    Server @ Pizza Antica Server Pizza Antica  ● S...
24471    Server @ Exotic Thai Cafe Server Exotic Thai C...
84001    Server @ Waraku Server Waraku We are a newly o...
84062    Server @ Kenta Ramen Server Kenta Ramen  New R...
Name: text, dtype: object

In [ ]:
# slightly more complex function that includes preprocessing with Spacy
# TFIDF transformation and embeddings

def get_tfidf_vector(word2vec_model, query):

    query = nlp(query, disable=["tagger", "parser", "ner"])
    query = [token.lemma_.lower() for token in query 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]
    if len(query) >= 1:
      words = tfidf.transform([query])
      return words @ word2vec_model.wv.vectors
    else:
        return []

In [ ]:
query = get_tfidf_vector(w2v_model, 'web developer and designer')
sims = cosine_similarity(query.reshape(1,300), df_jobs_w2v_tfidf)
ix = np.flip(np.argsort(sims)).tolist()[0][:10]
df_jobs['text'][ix]

59799    Tableau UI Developer Tableau UI Developer nan ...
81172    Jr. Ruby on Rails Developer @ ConsultNet Jr. R...
19011    Web Designer (freelance) @ Creative Circle Web...
59440    Sr. UX Designer @ Creative Circle Sr. UX Desig...
32658    Web Developer @ Creative Circle Web Developer ...
41978    Web designer/Front-end developer @ Collabera I...
79576    Web Developer @ Creative Circle Web Developer ...
34535    Graphic Designer (Print + Web) @ Creative Circ...
73957    Web Developer @ Creative Circle Web Developer ...
27721    Front End Developer @ ConsultNet Front End Dev...
Name: text, dtype: object

Adding Annoy - Approximate nearest neighbor matching

![](https://camo.githubusercontent.com/a056535a8490b4b1aa933808e77207276235a209e97a980119d3e438897e1d36/68747470733a2f2f7261772e6769746875622e636f6d2f73706f746966792f616e6e6f792f6d61737465722f616e6e2e706e67)

calculating cosines is great for small datasets (like ours) that gets more problematic once things get larger - here on-disk ANN approximation is the solution (we are talking 1M+ observations - when the vectorized matrices get too large)

I work easilly with colelctions of 40+ million with annoy...runs quick :-)

check out: https://github.com/spotify/annoy

In [ ]:
!pip install -q annoy

     |████████████████████████████████| 646 kB 22.1 MB/s 


In [ ]:
from annoy import AnnoyIndex

# instatiate a search tree (with shape n/300)
t = AnnoyIndex(df_jobs_w2v_tfidf.shape[1], 'angular') 

In [ ]:
# we will build that on disk (can reuse later if we store it somwhere)

t.on_disk_build('jobs_search_tree.annoy')

True

In [ ]:
# now we add all our vectors - line by line to the tree
# along with an index (here i - running index)
for i in tqdm.tqdm(range(df_jobs_w2v_tfidf.shape[0]),position=0, leave=True):
    t.add_item(i, df_jobs_w2v_tfidf[i])

100%|██████████| 84090/84090 [00:05<00:00, 14927.03it/s]


In [ ]:
# now we build the search tree (that creates partitions within the data-a bit like clustering)
# thereafter search will be performed within the nearest partitions (that reduces search time A LOT)
t.build(50, n_jobs=-1)

True

In [ ]:
t.get_nns_by_vector(df_jobs_w2v_tfidf[0], n=10, include_distances=True)

([0, 84057, 13448, 84044, 13453, 54458, 84081, 24471, 84001, 84062],
 [0.0,
  0.443498432636261,
  0.471523642539978,
  0.4842592477798462,
  0.4921287000179291,
  0.49800676107406616,
  0.49911630153656006,
  0.5047510266304016,
  0.5048704743385315,
  0.514162003993988])

In [ ]:
# now we can search by index
knn_search = t.get_nns_by_item(0, n=10, include_distances=True)
df_jobs['text'][knn_search[0]]

0        Server @ Tacolicious Server Tacolicious Tacoli...
84057    Server @ Kabuto Restaurant Server Kabuto Resta...
13448    Server @ BALEENkitchen Server BALEENkitchen  ●...
84044    Server @ Yuzu Server Yuzu  Yuzu is one of the ...
13453    Server @ Gonpachi Server Gonpachi  ● Serve foo...
54458    Server @ Sakae Sushi Server Sakae Sushi  Locat...
84081    Server @ Pizza Antica Server Pizza Antica  ● S...
24471    Server @ Exotic Thai Cafe Server Exotic Thai C...
84001    Server @ Waraku Server Waraku We are a newly o...
84062    Server @ Kenta Ramen Server Kenta Ramen  New R...
Name: text, dtype: object

In [ ]:
# now we can search by vector
knn_search = t.get_nns_by_vector(df_jobs_w2v_tfidf[0], n=10, include_distances=True)
df_jobs['text'][knn_search[0]]

0        Server @ Tacolicious Server Tacolicious Tacoli...
84057    Server @ Kabuto Restaurant Server Kabuto Resta...
13448    Server @ BALEENkitchen Server BALEENkitchen  ●...
84044    Server @ Yuzu Server Yuzu  Yuzu is one of the ...
13453    Server @ Gonpachi Server Gonpachi  ● Serve foo...
54458    Server @ Sakae Sushi Server Sakae Sushi  Locat...
84081    Server @ Pizza Antica Server Pizza Antica  ● S...
24471    Server @ Exotic Thai Cafe Server Exotic Thai C...
84001    Server @ Waraku Server Waraku We are a newly o...
84062    Server @ Kenta Ramen Server Kenta Ramen  New R...
Name: text, dtype: object

In [ ]:
# also with a free query
query = get_tfidf_vector(w2v_model, 'web developer and designer')

knn_search = t.get_nns_by_vector(query[0], n=10, include_distances=True) 
# need to index [0] our matrix - annoy likes vectors - that's opposite from sklearn cosine

df_jobs['text'][knn_search[0]]

59799    Tableau UI Developer Tableau UI Developer nan ...
81172    Jr. Ruby on Rails Developer @ ConsultNet Jr. R...
19011    Web Designer (freelance) @ Creative Circle Web...
59440    Sr. UX Designer @ Creative Circle Sr. UX Desig...
32658    Web Developer @ Creative Circle Web Developer ...
41978    Web designer/Front-end developer @ Collabera I...
79576    Web Developer @ Creative Circle Web Developer ...
34535    Graphic Designer (Print + Web) @ Creative Circ...
73957    Web Developer @ Creative Circle Web Developer ...
27721    Front End Developer @ ConsultNet Front End Dev...
Name: text, dtype: object